# Choice Set space for Neural Data

 The idea is to create a choice set that: samples the same SV difference (+ - some variation) with different trials  

In [ ]:
#Import the relevant libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows = 999

# 1. Input: patient's alpha and beta parameters

In [ ]:
#Define files & data pathway
patient_code = 'patient1_calibration'
patient_split_dir = '/Volumes/UCDN/Santiago_Guardo/data/split'
utility_dir = '/Volumes/UCDN/Santiago_Guardo/data/utility'
utility_analysis = os.path.join(utility_dir,'utility_CRDM_analysis.csv')
parameters = pd.read_csv('utility_analysis')

#Define the parameter values for the given participant. 
filtered_parameters = parameters[parameters['subject'] == patient_code]
alpha = filtered_parameters['alpha'].values[0]
beta = filtered_parameters['beta'].values[0]
gamma = filtered_parameters['gamma'].values[0]

print ("alpha: " + str(alpha))
print ("beta: " + str(beta))
print ("gamma: " + str(gamma))

